In [112]:
import pandas as pd
import numpy as np
import scipy.stats as st

In [2]:
bugs=pd.read_csv('ml-bugs.csv')

In [3]:
bugs.head()

,Species,Color,Length (mm)
0,Mobug,Brown,11.6
1,Mobug,Blue,16.3
2,Lobug,Blue,15.1
3,Lobug,Green,23.7
4,Lobug,Blue,18.4


In [120]:
def entropy(ar):
    ar=np.array(ar)
    count_total=ar.sum()
    return np.round(-np.sum(np.log2(ar/count_total)*(ar/count_total)),4)
entropy([1,2])==np.round(st.entropy([1,2],base=2),4)

True

In [89]:
def entropy_df(df): return entropy(df.groupby('Species')['Species'].count())

In [90]:
parent_entropy=entropy_df(bugs)
parent_entropy

0.9799

In [142]:
color_entropy=bugs.groupby('Color').apply(entropy_df)
color_weights=bugs.groupby('Color')['Species'].count()/len(bugs)

color_info_gain=np.round(parent_entropy-np.sum(color_entropy*color_weights),4)
print(f'Information gain by Color: {color_info_gain}')
print('color_entropy:\n',color_entropy)
print('\ncolor_weights:\n',color_weights)

Information gain by Color: 0.0753
color_entropy:
 Color
Blue     0.9710
Brown    0.9183
Green    0.8113
dtype: float64

color_weights:
 Color
Blue     0.416667
Brown    0.250000
Green    0.333333
Name: Species, dtype: float64


In [156]:
def length_info_calc(mask):
    bugs['condition']=np.where(mask,'small','big')

    length_entropy=bugs.groupby('condition').apply(entropy_df)
    length_weights=bugs.groupby('condition')['Species'].count()/len(bugs)

    len_info_gain=np.round(parent_entropy-np.sum(length_entropy*length_weights),4)
#     print(f'Information gain by Length: {len_info_gain}')
#     print('length_entropy:\n',length_entropy)
#     print('\nlength_weights:\n',length_weights)
    return len_info_gain

blue_info_gain=length_info_calc(bugs['Color']=='Blue')
brown_info_gain=length_info_calc(bugs['Color']=='Brown')
green_info_gain=length_info_calc(bugs['Color']=='Green')

len_info_gain_17=length_info_calc(bugs['Length (mm)']<17)
len_info_gain_20=length_info_calc(bugs['Length (mm)']<20)

### As grouping by length<17, gives maximum information gain, we consider that

In [157]:
print(blue_info_gain,green_info_gain,brown_info_gain,len_info_gain_20,len_info_gain_17)
blue_info_gain<green_info_gain<brown_info_gain<len_info_gain_20<len_info_gain_17

0.0006 0.0428 0.0616 0.1008 0.1127


True